# 🧹 Notebook 2: Pré-processamento de Dados

Este notebook demonstra o processo de pré-processamento dos dados coletados do Reddit, incluindo limpeza de texto, lematização e preparação para análise de sentimento.

## 🎯 Objetivos
- Carregar dados brutos coletados
- Aplicar limpeza e normalização de texto
- Realizar lematização usando SpaCy
- Filtrar dados por qualidade e relevância
- Salvar dados processados para próximas etapas
- Gerar estatísticas de processamento

## 📋 Pré-requisitos
- Dados coletados na etapa anterior
- Modelo SpaCy PT-BR instalado: `python -m spacy download pt_core_news_sm`
- Bibliotecas de processamento de texto instaladas

In [ ]:
# Imports e configuração inicial
import sys
import os
import pandas as pd
import numpy as np
import yaml
from datetime import datetime
from pathlib import Path
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Adicionar src ao path
sys.path.append('../src')

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configurar visualizações
plt.style.use('default')
sns.set_palette("husl")

# Verificar estrutura do projeto
project_root = Path("..").resolve()
print(f"📁 Diretório do projeto: {project_root}")
print(f"📂 Estrutura principal:")
for item in ["config", "src", "data", "scripts"]:
    path = project_root / item
    status = "✅" if path.exists() else "❌"
    print(f"  {status} {item}/")

print("\n🔧 Python path:")
for p in sys.path[-3:]:
    print(f"  - {p}")

In [ ]:
# Carregar configuração
config_path = project_root / "config" / "topic.yaml"

try:
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    print("📋 Configuração carregada:")
    print(f"  🎯 Tópico: {config['topic']}")
    print(f"  🔍 Keywords: {config['keywords'][:5]}...")  # Mostrar apenas primeiras 5
    print(f"  🔧 Filtros:")
    print(f"    - Comprimento: {config['filters']['min_length']}-{config['filters']['max_length']}")
    print(f"    - Idioma: {config['filters']['language']}")
    print(f"    - Score mínimo: {config['filters']['min_score']}")
    
except FileNotFoundError:
    print(f"❌ Arquivo de configuração não encontrado: {config_path}")
    print("💡 Execute: cp config/topic.yaml.example config/topic.yaml")
except Exception as e:
    print(f"❌ Erro ao carregar configuração: {e}")

In [ ]:
# Testar importação das bibliotecas de preprocessamento
print("🔍 Testando importação das bibliotecas de pré-processamento:")

try:
    from preprocessing.cleaner import TextCleaner
    print("✅ TextCleaner importado com sucesso")
except ImportError as e:
    print(f"❌ Erro ao importar TextCleaner: {e}")

try:
    import spacy
    # Testar se o modelo PT-BR está disponível
    nlp = spacy.load("pt_core_news_sm")
    print("✅ SpaCy PT-BR disponível")
except OSError:
    print("❌ Modelo SpaCy PT-BR não encontrado")
    print("💡 Instale com: python -m spacy download pt_core_news_sm")
except ImportError as e:
    print(f"❌ Erro ao importar SpaCy: {e}")
    print("💡 Instale com: pip install spacy")

try:
    from langdetect import detect
    print("✅ langdetect disponível")
except ImportError as e:
    print(f"❌ Erro ao importar langdetect: {e}")
    print("💡 Instale com: pip install langdetect")

# Verificar se DuckDB está disponível para otimização de parquet
try:
    import duckdb
    print("✅ DuckDB disponível (otimização de parquet)")
except ImportError:
    print("⚠️  DuckDB não disponível - usará pandas para parquet")
    print("💡 Para melhor performance: pip install duckdb")

## 📊 Exploração dos Dados Brutos

Vamos primeiro examinar os dados coletados para entender sua estrutura e qualidade.

In [ ]:
# Descobrir arquivos de dados brutos
raw_data_dir = project_root / "data" / "raw"

print(f"📁 Buscando dados em: {raw_data_dir}")

if not raw_data_dir.exists():
    print("❌ Diretório de dados brutos não encontrado")
    print("💡 Execute primeiro o notebook de coleta (01_coleta.ipynb)")
else:
    # Buscar por arquivos CSV
    csv_files = list(raw_data_dir.rglob("*.csv"))
    
    if not csv_files:
        print("❌ Nenhum arquivo CSV encontrado")
        print("💡 Execute primeiro a coleta de dados")
    else:
        print(f"📄 Encontrados {len(csv_files)} arquivo(s) CSV:")
        
        for i, file in enumerate(csv_files, 1):
            size_mb = file.stat().st_size / (1024*1024)
            modified = datetime.fromtimestamp(file.stat().st_mtime)
            print(f"  {i}. {file.name}")
            print(f"     📏 Tamanho: {size_mb:.2f} MB")
            print(f"     📅 Modificado: {modified}")
            
        # Selecionar arquivo mais recente por padrão
        latest_file = max(csv_files, key=lambda x: x.stat().st_mtime)
        print(f"\n🎯 Usando arquivo mais recente: {latest_file.name}")
        
        # Carregar preview dos dados
        try:
            df_preview = pd.read_csv(latest_file, nrows=5)
            print(f"\n📊 Preview dos dados ({latest_file.name}):")
            print(f"Colunas: {list(df_preview.columns)}")
            
            # Mostrar informações do dataset completo
            df_full = pd.read_csv(latest_file)
            print(f"\n📈 Estatísticas do dataset:")
            print(f"  📊 Total de registros: {len(df_full):,}")
            print(f"  📝 Comprimento médio do texto: {df_full['text'].str.len().mean():.1f} caracteres")
            print(f"  📅 Período dos dados: {df_full['timestamp'].min()} até {df_full['timestamp'].max()}")
            
            if 'subreddit' in df_full.columns:
                subreddit_counts = df_full['subreddit'].value_counts().head(5)
                print(f"  🔗 Top 5 subreddits: {subreddit_counts.to_dict()}")
                
            selected_file = latest_file
            
        except Exception as e:
            print(f"❌ Erro ao carregar dados: {e}")
            selected_file = None

In [ ]:
# Análise exploratória dos dados brutos
if 'selected_file' in locals() and selected_file:
    print("🔍 Realizando análise exploratória dos dados...")
    
    # Carregar dados completos
    df_raw = pd.read_csv(selected_file)
    
    print(f"\n📊 Informações gerais:")
    print(f"  Registros: {len(df_raw):,}")
    print(f"  Colunas: {len(df_raw.columns)}")
    print(f"  Memória usada: {df_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Análise de qualidade dos dados
    print(f"\n🔍 Qualidade dos dados:")
    print(f"  Valores nulos por coluna:")
    null_counts = df_raw.isnull().sum()
    for col in null_counts.index:
        if null_counts[col] > 0:
            print(f"    {col}: {null_counts[col]} ({null_counts[col]/len(df_raw)*100:.1f}%)")
    
    # Análise de comprimento de texto
    text_lengths = df_raw['text'].str.len()
    print(f"\n📏 Distribuição do comprimento do texto:")
    print(f"  Mínimo: {text_lengths.min()} caracteres")
    print(f"  Médio: {text_lengths.mean():.1f} caracteres")
    print(f"  Mediano: {text_lengths.median():.1f} caracteres")
    print(f"  Máximo: {text_lengths.max()} caracteres")
    
    # Visualização da distribuição de comprimento
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(text_lengths, bins=50, alpha=0.7, color='skyblue')
    plt.xlabel('Comprimento do texto (caracteres)')
    plt.ylabel('Frequência')
    plt.title('Distribuição do Comprimento do Texto')
    plt.axvline(text_lengths.mean(), color='red', linestyle='--', label=f'Média: {text_lengths.mean():.0f}')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    if 'score' in df_raw.columns:
        plt.scatter(text_lengths, df_raw['score'], alpha=0.5, s=1)
        plt.xlabel('Comprimento do texto')
        plt.ylabel('Score do post')
        plt.title('Comprimento vs Score')
    
    plt.tight_layout()
    plt.show()
    
    # Exemplos de texto
    print(f"\n📝 Exemplos de textos:")
    sample_texts = df_raw['text'].sample(3).tolist()
    for i, text in enumerate(sample_texts, 1):
        print(f"\n  Exemplo {i} ({len(text)} chars):")
        print(f"  {text[:200]}{'...' if len(text) > 200 else ''}")
        
else:
    print("❌ Nenhum arquivo de dados selecionado")

## 🧹 Pré-processamento com TextCleaner

Agora vamos aplicar o pré-processamento usando a classe TextCleaner.

In [ ]:
# Inicializar o TextCleaner e processar uma amostra
if 'df_raw' in locals() and len(df_raw) > 0:
    try:
        # Inicializar TextCleaner
        text_cleaner = TextCleaner(config)
        print("✅ TextCleaner inicializado com sucesso")
        
        # Testar processamento com uma amostra pequena primeiro
        print("\n🧪 Testando processamento com amostra de 10 textos...")
        sample_df = df_raw.sample(n=min(10, len(df_raw))).copy()
        
        # Processar amostra
        processed_sample = text_cleaner.process_dataframe(sample_df, 'text')
        
        print(f"✅ Processamento de amostra concluído")
        print(f"  📊 Registros originais: {len(sample_df)}")
        print(f"  📊 Registros válidos: {len(processed_sample)}")
        
        # Mostrar exemplo de processamento
        if len(processed_sample) > 0:
            print("\n📝 Exemplo de processamento:")
            example = processed_sample.iloc[0]
            print(f"\n  Original: {example['original'][:200]}...")
            print(f"\n  Limpo: {example['cleaned'][:200]}...")
            print(f"\n  Lematizado: {example['lemmatized'][:200]}...")
            print(f"\n  Comprimento: Original={len(example['original'])}, Limpo={len(example['cleaned'])}, Lematizado={len(example['lemmatized'])}")
        
        # Estatísticas do processamento
        stats = text_cleaner.get_processing_stats(processed_sample)
        print(f"\n📈 Estatísticas do processamento:")
        for key, value in stats.items():
            print(f"  {key}: {value}")
            
    except Exception as e:
        print(f"❌ Erro no processamento: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Dados brutos não disponíveis")

In [ ]:
# Processamento completo dos dados
if 'text_cleaner' in locals() and 'df_raw' in locals():
    print(f"🚀 Iniciando processamento completo de {len(df_raw):,} registros...")
    print("⏱️  Este processo pode levar alguns minutos...")
    
    try:
        # Processar todo o dataset
        start_time = datetime.now()
        
        df_processed = text_cleaner.process_dataframe(df_raw, 'text')
        
        end_time = datetime.now()
        processing_time = (end_time - start_time).total_seconds()
        
        print(f"✅ Processamento concluído em {processing_time:.1f} segundos")
        print(f"  📊 Registros processados: {len(df_raw):,}")
        print(f"  📊 Registros válidos: {len(df_processed):,}")
        print(f"  📊 Taxa de aprovação: {len(df_processed)/len(df_raw)*100:.1f}%")
        print(f"  ⚡ Velocidade: {len(df_raw)/processing_time:.1f} textos/segundo")
        
        # Estatísticas detalhadas
        final_stats = text_cleaner.get_processing_stats(df_processed)
        print(f"\n📈 Estatísticas finais:")
        for key, value in final_stats.items():
            if isinstance(value, float):
                print(f"  {key}: {value:.1f}")
            else:
                print(f"  {key}: {value}")
        
        # Comparação antes/depois
        print(f"\n📊 Comparação antes/depois:")
        print(f"  Comprimento médio original: {df_raw['text'].str.len().mean():.1f} → {df_processed['cleaned'].str.len().mean():.1f}")
        print(f"  Registros: {len(df_raw):,} → {len(df_processed):,} ({len(df_processed)/len(df_raw)*100:.1f}%)")
        
        # Visualização da distribuição de comprimentos processados
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.hist(df_raw['text'].str.len(), bins=50, alpha=0.7, label='Original', color='lightcoral')
        plt.xlabel('Comprimento (caracteres)')
        plt.ylabel('Frequência')
        plt.title('Texto Original')
        plt.legend()
        
        plt.subplot(1, 3, 2)
        plt.hist(df_processed['cleaned'].str.len(), bins=50, alpha=0.7, label='Limpo', color='lightgreen')
        plt.xlabel('Comprimento (caracteres)')
        plt.ylabel('Frequência')
        plt.title('Texto Limpo')
        plt.legend()
        
        plt.subplot(1, 3, 3)
        plt.hist(df_processed['lemmatized'].str.len(), bins=50, alpha=0.7, label='Lematizado', color='lightblue')
        plt.xlabel('Comprimento (caracteres)')
        plt.ylabel('Frequência')
        plt.title('Texto Lematizado')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Erro no processamento completo: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ TextCleaner ou dados brutos não disponíveis")

## 💾 Salvando Dados Processados

Vamos salvar os dados processados em formato Parquet para uso posterior.

In [ ]:
# Salvar dados processados
if 'df_processed' in locals() and len(df_processed) > 0:
    # Criar diretório de dados processados
    processed_dir = project_root / "data" / "processed"
    processed_dir.mkdir(parents=True, exist_ok=True)
    
    # Gerar nome do arquivo com timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    topic_slug = config['topic'].lower().replace(' ', '_').replace('-', '_')
    
    # Salvar em diferentes formatos
    parquet_file = processed_dir / f"processed_{topic_slug}_{timestamp}.parquet"
    csv_file = processed_dir / f"processed_{topic_slug}_{timestamp}.csv"
    
    try:
        print(f"💾 Salvando dados processados...")
        
        # Salvar em Parquet (formato eficiente)
        df_processed.to_parquet(parquet_file, index=False, compression='snappy')
        parquet_size = parquet_file.stat().st_size / (1024*1024)
        print(f"✅ Parquet salvo: {parquet_file.name} ({parquet_size:.2f} MB)")
        
        # Salvar em CSV (para compatibilidade)
        df_processed.to_csv(csv_file, index=False)
        csv_size = csv_file.stat().st_size / (1024*1024)
        print(f"✅ CSV salvo: {csv_file.name} ({csv_size:.2f} MB)")
        
        print(f"\n📊 Compressão: CSV {csv_size:.1f} MB → Parquet {parquet_size:.1f} MB ({parquet_size/csv_size*100:.1f}%)")
        
        # Salvar metadados
        metadata = {
            'processed_at': datetime.now().isoformat(),
            'original_file': str(selected_file.name) if 'selected_file' in locals() else 'unknown',
            'original_records': len(df_raw) if 'df_raw' in locals() else 0,
            'processed_records': len(df_processed),
            'processing_time_seconds': processing_time if 'processing_time' in locals() else 0,
            'config': config,
            'statistics': final_stats if 'final_stats' in locals() else {}
        }
        
        metadata_file = processed_dir / f"metadata_{topic_slug}_{timestamp}.yaml"
        with open(metadata_file, 'w', encoding='utf-8') as f:
            yaml.dump(metadata, f, default_flow_style=False, allow_unicode=True)
        
        print(f"✅ Metadados salvos: {metadata_file.name}")
        
        print(f"\n🎯 Arquivos gerados:")
        print(f"  📄 {parquet_file.name} - Dados processados (recomendado)")
        print(f"  📄 {csv_file.name} - Dados processados (CSV)")
        print(f"  📄 {metadata_file.name} - Metadados do processamento")
        
        # Verificar integridade dos dados salvos
        print(f"\n🔍 Verificando integridade...")
        df_test = pd.read_parquet(parquet_file)
        assert len(df_test) == len(df_processed), "Erro na verificação de integridade"
        print(f"✅ Integridade verificada - {len(df_test):,} registros")
        
    except Exception as e:
        print(f"❌ Erro ao salvar dados: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("❌ Nenhum dado processado disponível para salvar")

## 🚀 Execução via Script

Alternativamente, você pode executar todo o pré-processamento usando o script dedicado:

In [ ]:
# Executar pré-processamento usando o script preprocess.py
import subprocess

if 'selected_file' in locals() and selected_file:
    # Diretório do arquivo selecionado
    input_dir = selected_file.parent
    
    # Comando para executar o script de preprocessamento
    cmd = [
        "python", 
        str(project_root / "scripts" / "preprocess.py"),
        "--config", str(project_root / "config" / "topic.yaml"),
        "--input-dir", str(input_dir),
        "--output-dir", str(project_root / "data" / "processed"),
        "--format", "parquet",
        "--hash-users",
        "--combine",
        "--verbose"
    ]
    
    print(f"🔧 Comando do script:")
    print(f"   {' '.join(cmd)}")
    print("\n" + "="*50)
    print("🚀 EXECUTANDO PRÉ-PROCESSAMENTO VIA SCRIPT...")
    print("="*50)
    
    try:
        # Executar o comando
        result = subprocess.run(cmd, capture_output=True, text=True, cwd=str(project_root))
        
        print("📤 SAÍDA:")
        print(result.stdout)
        
        if result.stderr:
            print("⚠️  ERROS/AVISOS:")
            print(result.stderr)
        
        if result.returncode == 0:
            print("✅ Pré-processamento concluído com sucesso!")
            
            # Verificar arquivos gerados
            processed_dir = project_root / "data" / "processed"
            if processed_dir.exists():
                files = list(processed_dir.glob("*.parquet"))
                print(f"📄 Arquivos processados gerados: {len(files)}")
                for file in files:
                    size_mb = file.stat().st_size / (1024*1024)
                    print(f"  - {file.name}: {size_mb:.2f} MB")
                    
                    # Verificar conteúdo
                    try:
                        df_check = pd.read_parquet(file)
                        print(f"    📊 {len(df_check):,} registros processados")
                        if 'lemmatized' in df_check.columns:
                            avg_length = df_check['lemmatized'].str.len().mean()
                            print(f"    📏 Comprimento médio (lematizado): {avg_length:.1f} caracteres")
                    except Exception as e:
                        print(f"    ⚠️  Erro ao verificar arquivo: {e}")
        else:
            print(f"❌ Pré-processamento falhou com código: {result.returncode}")
            
    except Exception as e:
        print(f"❌ Erro ao executar script: {e}")
    
    print("\n" + "="*50)
    
else:
    print("❌ Nenhum arquivo de dados selecionado para processar")
    print("💡 Execute primeiro as células de descoberta de dados")

## 📊 Resumo do Pré-processamento

Este notebook realizou as seguintes etapas de pré-processamento:

### ✅ **Etapas Executadas:**
1. **Limpeza de texto**: Remoção de URLs, mentions, hashtags e caracteres especiais
2. **Normalização**: Conversão para minúsculas e remoção de espaços extras
3. **Lematização**: Redução das palavras às suas formas básicas usando SpaCy PT-BR
4. **Filtragem**: Aplicação de filtros de comprimento, idioma e qualidade
5. **Validação**: Verificação de integridade e qualidade dos dados processados

### 📈 **Próximos Passos:**
- **Análise Exploratória**: Examine os dados processados em detalhes
- **Rotulagem**: Prepare dados para treinamento de modelos
- **Modelagem**: Treine modelos de análise de sentimento
- **Avaliação**: Teste e valide os modelos treinados

### 📁 **Arquivos Gerados:**
- `processed_*.parquet`: Dados processados em formato eficiente
- `processed_*.csv`: Dados processados em formato CSV
- `metadata_*.yaml`: Metadados do processamento

Os dados estão prontos para as próximas etapas do pipeline de análise de sentimento! 🎉